In [1]:


    balance = ticker.get_balance_sheet()

    current_ratio = info.get('currentRatio', 'N/A')
    roa = info.get('returnOnAssets', 'N/A')
    roe = info.get('returnOnEquity', 'N/A')
    long_term_debt = balance.at['LongTermDebt', '2022-12-31']
    total_assets = balance.at['TotalAssets', '2022-12-31']
    
    long_assets = long_term_debt / total_assets

    data_list.append({
        'Symbol': symbol,
        'Current Ratio': current_ratio,
        'ROA': roa,
        'ROE': roe,
        'Long Term Debt': long_term_debt,
        'Total Assets': total_assets,
        'Long Term Debt / Total Assets': long_assets

IndentationError: unexpected indent (1316063514.py, line 3)

In [2]:
import yfinance as yf

In [6]:
ticker = yf.Ticker('BAC')
info = ticker.get_info()

HTTPError: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v6/finance/quoteSummary/BAC?modules=financialData&modules=quoteType&modules=defaultKeyStatistics&modules=assetProfile&modules=summaryDetail&ssl=true